In [1]:
import numpy as np
import librosa
import moviepy.editor as mp
import os
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv("data/MELD/train_sent_emo.csv")

In [4]:
data

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my company’s tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You must’ve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So let’s talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
9984,10474,You or me?,Chandler,neutral,neutral,1038,13,2,3,"00:00:48,173","00:00:50,799"
9985,10475,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,neutral,neutral,1038,14,2,3,"00:00:51,009","00:00:53,594"
9986,10476,"You guys are messing with me, right?",Joey,surprise,positive,1038,15,2,3,"00:01:00,518","00:01:03,520"
9987,10477,Yeah.,All,neutral,neutral,1038,16,2,3,"00:01:05,398","00:01:07,274"


In [8]:
invalid = [(125, 3), (795, 0)]

In [48]:
df = pd.DataFrame(columns=['feature'])

# loop feature extraction over the entire dataset
counter=0
speakers = []
emotions = []
sentiments = []
for i in range(len(data)):
    entry = data.iloc[i]
    dia = entry['Dialogue_ID']
    utt = entry['Utterance_ID']
    tup = (dia, utt)
    if tup in invalid:
        continue
    X, sample_rate = librosa.load(f'train_splits/dia{dia}_utt{utt}.wav'.format(dia, utt)
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )
    sample_rate = np.array(sample_rate)
    try:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    except ValueError:
        continue
    df.loc[counter] = [mfccs]
    speakers.append(data['Speaker'].iloc[i])
    emotions.append(data['Emotion'].iloc[i])
    sentiments.append(data['Sentiment'].iloc[i])
    counter=counter+1   

# Check a few records to make sure its processed successfully
print(len(df))
df.head()

9689


,feature
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314..."
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955..."
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,..."
3,"[-4.921062, -7.6697946, -11.833557, -11.440711..."
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,..."


In [16]:
df[df.notnull()].reset_index(drop = True)

,feature
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314..."
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955..."
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,..."
3,"[-4.921062, -7.6697946, -11.833557, -11.440711..."
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,..."
...,...
9982,"[-16.539276, -18.343447, -20.658241, -21.98034..."
9983,"[-7.0690284, -11.294577, -16.206984, -17.69017..."
9984,"[-14.950315, -15.817601, -19.381824, -23.26894..."
9985,"[-16.07852, -17.39303, -21.3409, -21.931488, -..."


In [49]:
df

,feature
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314..."
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955..."
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,..."
3,"[-4.921062, -7.6697946, -11.833557, -11.440711..."
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,..."
...,...
9684,"[-16.539276, -18.343447, -20.658241, -21.98034..."
9685,"[-7.0690284, -11.294577, -16.206984, -17.69017..."
9686,"[-14.950315, -15.817601, -19.381824, -23.26894..."
9687,"[-16.07852, -17.39303, -21.3409, -21.931488, -..."


In [17]:
df['label'] = emotions
df.head()

,feature,label
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314...",neutral
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955...",neutral
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,...",neutral
3,"[-4.921062, -7.6697946, -11.833557, -11.440711...",neutral
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,...",surprise


In [21]:
x = np.array(df['feature'].values.tolist())

In [19]:
df2 = df[df['feature'].notnull()]

In [20]:
df2

,feature,label
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314...",neutral
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955...",neutral
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,...",neutral
3,"[-4.921062, -7.6697946, -11.833557, -11.440711...",neutral
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,...",surprise
...,...,...
9982,"[-16.539276, -18.343447, -20.658241, -21.98034...",neutral
9983,"[-7.0690284, -11.294577, -16.206984, -17.69017...",neutral
9984,"[-14.950315, -15.817601, -19.381824, -23.26894...",surprise
9985,"[-16.07852, -17.39303, -21.3409, -21.931488, -...",neutral


In [73]:
main_speaker = ['Ross', 'Rachel', 'Joey', 'Chandler', 'Monica', 'Phoebe']

In [75]:
df2[(df2['label'] == 'Ross') | (df2['label'] == 'Rachel') | (df2['label'] == 'Joey') | (df2['label'] == 'Chandler') | (df2['label'] == 'Monica') | (df2['label'] == 'Phoebe')]

,feature,label
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314...",Chandler
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,...",Chandler
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,...",Chandler
6,"[-12.849821, -14.572752, -15.051802, -17.70573...",Chandler
8,"[-9.331165, -8.034653, -10.06244, -11.451357, ...",Chandler
...,...,...
9981,"[-16.344809, -15.342422, -16.601727, -16.62337...",Joey
9982,"[-16.539276, -18.343447, -20.658241, -21.98034...",Chandler
9983,"[-7.0690284, -11.294577, -16.206984, -17.69017...",Ross
9984,"[-14.950315, -15.817601, -19.381824, -23.26894...",Joey


In [80]:
final_data = pd.DataFrame(speakers)
final_data

,0
0,Chandler
1,The Interviewer
2,Chandler
3,The Interviewer
4,Chandler
...,...
9684,Chandler
9685,Ross
9686,Joey
9687,All


In [44]:
df = df[df['feature'].notnull()].reset_index(drop = True)

In [51]:
pd.concat([final_data ,pd.DataFrame(df['feature'].values.tolist())],axis=1)

,0,0,1,2,3,4,5,6,7,8,...,206,207,208,209,210,211,212,213,214,215
0,neutral,-4.118611,-3.882613,-7.018807,-6.355331,-6.485232,-7.660092,-6.798985,-5.984684,-4.585024,...,-15.647587,-18.681343,-17.999174,-18.979235,-16.262184,-17.446283,-15.761398,-10.201248,-8.537461,-7.215563
1,neutral,-6.545803,-6.421562,-6.115737,-7.164955,-5.427852,-4.090354,-5.502530,-8.139649,-9.480793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,neutral,-7.289024,-7.285092,-7.915808,-9.007195,-8.910819,-7.923785,-9.262099,-10.494543,-13.232133,...,-18.432789,-17.414785,-20.880537,-32.400608,-49.170994,-49.526024,NaN,NaN,NaN,NaN
3,neutral,-4.921062,-7.669795,-11.833557,-11.440711,-10.774164,-10.647370,-10.992935,-10.366111,-11.788311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,surprise,-7.236552,-6.116012,-5.304642,-4.556395,-4.673206,-5.744306,-7.449941,-9.951056,-10.398076,...,-13.783406,-14.083395,-16.860638,-14.512985,-13.407324,-13.554709,-14.235941,-14.484867,-13.809485,-10.920336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9684,neutral,-16.539276,-18.343447,-20.658241,-21.980347,-21.474989,-19.503977,-16.668499,-16.726593,-18.331608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9685,neutral,-7.069028,-11.294577,-16.206984,-17.690178,-19.098488,-17.921696,-16.861347,-17.653360,-18.888561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9686,surprise,-14.950315,-15.817601,-19.381824,-23.268942,-21.753664,-21.772388,-21.048897,-23.347900,-23.087860,...,-15.578559,-17.014122,-15.908967,-16.265728,-16.332285,-15.722894,-13.017822,-10.997358,-12.779449,-13.571667
9687,neutral,-16.078520,-17.393030,-21.340900,-21.931488,-20.448271,-19.997286,-20.872786,-18.105610,-18.547226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.tail()

,feature,label
9684,"[-16.539276, -18.343447, -20.658241, -21.98034...",neutral
9685,"[-7.0690284, -11.294577, -16.206984, -17.69017...",neutral
9686,"[-14.950315, -15.817601, -19.381824, -23.26894...",surprise
9687,"[-16.07852, -17.39303, -21.3409, -21.931488, -...",neutral
9688,"[-9.082689, -11.29994, -16.13089, -17.129896, ...",joy


In [89]:
final_data = final_data.iloc[df.index]

In [91]:
final_data = final_data.reset_index(drop =True)

In [81]:
final_data.columns = ['label']
final_data

,label
0,Chandler
1,The Interviewer
2,Chandler
3,The Interviewer
4,Chandler
...,...
9684,Chandler
9685,Ross
9686,Joey
9687,All


In [93]:
df = df.reset_index(drop = True)
df

,feature
0,"[-4.1186113, -3.8826127, -7.018807, -6.3553314..."
1,"[-6.5458026, -6.4215617, -6.1157374, -7.164955..."
2,"[-7.289024, -7.2850924, -7.9158077, -9.007195,..."
3,"[-4.921062, -7.6697946, -11.833557, -11.440711..."
4,"[-7.2365522, -6.1160116, -5.304642, -4.556395,..."
...,...
9684,"[-16.539276, -18.343447, -20.658241, -21.98034..."
9685,"[-7.0690284, -11.294577, -16.206984, -17.69017..."
9686,"[-14.950315, -15.817601, -19.381824, -23.26894..."
9687,"[-16.07852, -17.39303, -21.3409, -21.931488, -..."


In [82]:
final_data = pd.concat([final_data ,pd.DataFrame(df['feature'].values.tolist())],axis=1)

In [83]:
final_data = final_data.fillna(0)

In [86]:
final_data = final_data[(final_data['label'] == 'Ross') | (final_data['label'] == 'Rachel') | (final_data['label'] == 'Joey') | (final_data['label'] == 'Chandler') | (final_data['label'] == 'Monica') | (final_data['label'] == 'Phoebe')]

In [87]:
final_data.columns = ['label'] + list(range(0, 216))

In [88]:
final_data.head()

,label,0,1,2,3,4,5,6,7,8,...,206,207,208,209,210,211,212,213,214,215
0,Chandler,-4.118611,-3.882613,-7.018807,-6.355331,-6.485232,-7.660092,-6.798985,-5.984684,-4.585024,...,-15.647587,-18.681343,-17.999174,-18.979235,-16.262184,-17.446283,-15.761398,-10.201248,-8.537461,-7.215563
2,Chandler,-7.289024,-7.285092,-7.915808,-9.007195,-8.910819,-7.923785,-9.262099,-10.494543,-13.232133,...,-18.432789,-17.414785,-20.880537,-32.400608,-49.170994,-49.526024,0.000000,0.000000,0.000000,0.000000
4,Chandler,-7.236552,-6.116012,-5.304642,-4.556395,-4.673206,-5.744306,-7.449941,-9.951056,-10.398076,...,-13.783406,-14.083395,-16.860638,-14.512985,-13.407324,-13.554709,-14.235941,-14.484867,-13.809485,-10.920336
6,Chandler,-12.849821,-14.572752,-15.051802,-17.705736,-17.799690,-14.104472,-10.476247,-9.289321,-10.517152,...,-9.676641,-9.251141,-9.473232,-11.027970,-11.172906,-11.745040,-11.490341,-11.854862,-9.754216,-7.710693
8,Chandler,-9.331165,-8.034653,-10.062440,-11.451357,-10.947601,-12.337722,-12.559923,-12.427165,-9.983657,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [89]:
X_train, X_test, y_train, y_test = train_test_split(final_data.drop(['label'],axis=1)
                                                    , final_data.label
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )

In [90]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
8133,-14.356584,-15.019550,-19.316887,-23.168058,-22.441563,-23.412210,-23.453001,-16.679441,-12.898342,-12.541500,...,-24.679836,-23.432791,-24.895384,-25.781054,-25.103315,-26.665110,-27.330517,-26.185291,-26.143831,-25.642771
7078,-12.653582,-11.853458,-11.659241,-11.851955,-10.493947,-11.232570,-11.941900,-12.144562,-12.603795,-13.626214,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6605,-10.642035,-12.489892,-15.169722,-13.178637,-13.621235,-14.213543,-14.498265,-13.613256,-12.265909,-14.154568,...,-14.824059,-15.821795,-17.122978,-14.459431,-13.593650,-13.851318,-13.983979,-16.020365,-14.313599,-13.143091
6626,-9.864800,-13.193706,-18.895748,-19.444693,-18.394016,-17.674520,-19.184050,-19.921183,-20.572945,-20.207411,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6153,-14.247641,-15.166187,-17.055735,-16.990847,-17.149860,-16.348763,-15.504612,-16.866852,-18.224123,-18.459038,...,-15.004107,-15.235523,-14.507867,-14.388397,-14.985896,-14.279561,-14.387181,-13.718474,-13.959253,-12.223347


In [91]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
8133,0.387683,0.411352,0.013416,-0.576613,-0.468221,-0.604913,-0.600989,0.413259,0.978362,1.026854,...,-1.503059,-1.404344,-1.541351,-1.626436,-1.555812,-1.714370,-1.791743,-1.685248,-1.777151,-1.815084
7078,0.625623,0.897683,1.200094,1.171811,1.366430,1.259443,1.140281,1.097254,1.022567,0.864953,...,0.794461,0.781915,0.781720,0.781108,0.778991,0.770234,0.769063,0.767057,0.751074,0.729055
6605,0.906673,0.799923,0.656087,0.966829,0.886210,0.803141,0.753582,0.875731,1.073276,0.786092,...,-0.585555,-0.694245,-0.816082,-0.569175,-0.485324,-0.520406,-0.541203,-0.733282,-0.633115,-0.574933
6626,1.015267,0.691813,0.078678,-0.001324,0.153312,0.273364,0.044770,-0.075692,-0.173414,-0.117339,...,0.794461,0.781915,0.781720,0.781108,0.778991,0.770234,0.769063,0.767057,0.751074,0.729055
6153,0.402904,0.388827,0.363818,0.377813,0.344362,0.476299,0.601354,0.384992,0.179088,0.143618,...,-0.602316,-0.639546,-0.572058,-0.562541,-0.614813,-0.560308,-0.578983,-0.517705,-0.598848,-0.483681


In [92]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)

(6030, 216)
['Chandler' 'Joey' 'Monica' 'Phoebe' 'Rachel' 'Ross']


In [93]:
y_train

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [94]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

(6030, 216, 1)

In [95]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6)) # Target class number
#model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
#opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_32 (Conv1D)           (None, 216, 256)          2304      
_________________________________________________________________
activation_32 (Activation)   (None, 216, 256)          0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 216, 256)          524544    
_________________________________________________________________
batch_normalization_8 (Batch (None, 216, 256)          1024      
_________________________________________________________________
activation_33 (Activation)   (None, 216, 256)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 216, 256)          0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 27, 256)          

In [97]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
121/121 [==============================] - 40s 309ms/step - loss: 3.0909 - accuracy: 0.1774 - val_loss: 3.8842 - val_accuracy: 0.1795
Epoch 2/100
121/121 [==============================] - 37s 304ms/step - loss: 1.8113 - accuracy: 0.1919 - val_loss: 2.1028 - val_accuracy: 0.1974
Epoch 3/100
121/121 [==============================] - 36s 299ms/step - loss: 1.7729 - accuracy: 0.2245 - val_loss: 1.8523 - val_accuracy: 0.1865
Epoch 4/100
121/121 [==============================] - 38s 316ms/step - loss: 1.7485 - accuracy: 0.2415 - val_loss: 1.8033 - val_accuracy: 0.1790
Epoch 5/100
121/121 [==============================] - 39s 322ms/step - loss: 1.7443 - accuracy: 0.2438 - val_loss: 1.7932 - val_accuracy: 0.2029
Epoch 6/100
121/121 [==============================] - 41s 338ms/step - loss: 1.7307 - accuracy: 0.2532 - val_loss: 1.7719 - val_accuracy: 0.2208
Epoch 7/100
121/121 [==============================] - 46s 383ms/step - loss: 1.7274 - accuracy: 0.2506 - val_loss: 1.7787 -

Epoch 57/100
121/121 [==============================] - 55s 456ms/step - loss: 2.2925 - accuracy: 0.2630 - val_loss: 2.0597 - val_accuracy: 0.1895
Epoch 58/100
121/121 [==============================] - 67s 554ms/step - loss: 2.1396 - accuracy: 0.2690 - val_loss: 1.9936 - val_accuracy: 0.1845
Epoch 59/100
121/121 [==============================] - 66s 547ms/step - loss: 2.1584 - accuracy: 0.2569 - val_loss: 2.0402 - val_accuracy: 0.2123
Epoch 60/100
121/121 [==============================] - 62s 512ms/step - loss: 2.1644 - accuracy: 0.2539 - val_loss: 2.0744 - val_accuracy: 0.2024
Epoch 61/100
121/121 [==============================] - 57s 474ms/step - loss: 2.0424 - accuracy: 0.2720 - val_loss: 2.0477 - val_accuracy: 0.2019
Epoch 62/100
121/121 [==============================] - 58s 482ms/step - loss: 2.0156 - accuracy: 0.2734 - val_loss: 2.0736 - val_accuracy: 0.1964
Epoch 63/100
121/121 [==============================] - 54s 448ms/step - loss: 2.2833 - accuracy: 0.2632 - val_loss: 2